In [2]:
#Libraries
import pandas as pd
import datetime as dt

In [3]:
### RATES ###
df_rates=pd.read_csv('Rates_csv/rates_2021_2023.csv') # CHEQUEAR FUENTE DE ORIGEN

In [4]:
# Selecting columns & renaming
df_rates = df_rates[['feed_date','symbol','feed_price']] 
df_rates = df_rates.rename(columns={'feed_date':'date'})

In [13]:
df_rates['date'] = pd.to_datetime(df_rates['date']).dt.date
#df_rates['date'] = pd.to_datetime(df_rates['date'])

In [20]:
df_rates[(df_rates['date'] ==  '2021-01-05')].sort_values('date').head(20)

,date,symbol,feed_price


In [15]:
df_rates[(df_rates['date'] == '2023-10-15')].sort_values('date').head(20)

,date,symbol,feed_price


In [8]:
# !No hay continuidad de fechas

In [22]:
# Function to keep last register, per day

def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime type if it is not already
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' to ensure that the data is in chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
    return data_grouped

In [23]:
start_date = '2021-01-01'
end_date = '2023-10-21'

rates = get_closing_prices(df_rates, start_date, end_date)

In [24]:
rates

,symbol,date,feed_price
0,USDBRL,2021-01-01,5.1935
1,USDBRL,2021-01-02,5.1934
2,USDBRL,2021-01-04,5.2377
3,USDBRL,2021-01-05,5.3175
4,USDBRL,2021-01-06,5.3006
...,...,...,...
3940,USDPHP,2023-10-08,56.6985
3941,USDPHP,2023-10-11,56.7598
3942,USDPHP,2023-10-16,56.8120
3943,USDPHP,2023-10-19,56.7696


In [25]:
#Date format
#rates['date'] = rates['date'].dt.date

In [26]:
rates[rates['date'] > '2021-12-27'].sort_values('date')

,symbol,date,feed_price
280,USDBRL,2021-12-29,5.6289
1069,USDGTQ,2021-12-29,7.7313
3436,USDPHP,2021-12-29,50.8563
2647,USDMXN,2021-12-29,20.6566
1858,USDINR,2021-12-29,74.7689
...,...,...,...
2366,USDINR,2023-10-20,83.1572
1577,USDGTQ,2023-10-20,7.8408
788,USDBRL,2023-10-20,5.0494
3155,USDMXN,2023-10-20,18.2793


In [15]:
# Generating an empty df with date continuity and currency included in rates df
from itertools import product

def generate_continuous_dates(data):
    # Obtener la lista única de monedas y fechas disponibles
    countries = data['symbol'].unique()
    dates = pd.date_range(start=data['date'].min(), end=data['date'].max(), freq='D')

    # Generar todas las combinaciones de países y fechas continuas
    all_dates = list(product(countries, dates))
    continuous_dates = pd.DataFrame(all_dates, columns=['symbol', 'date'])

    return continuous_dates

In [16]:
rates_cont = generate_continuous_dates(rates)

In [17]:
rates_cont = pd.merge(rates_cont, rates, on=['symbol', 'date'], how='left')

In [18]:
rates_cont[rates_cont['date'] > '2021-12-27'].sort_values('date')

,symbol,date,feed_price
361,USDBRL,2021-12-28,NaN
3430,USDMXN,2021-12-28,NaN
2407,USDINR,2021-12-28,NaN
4453,USDPHP,2021-12-28,NaN
1384,USDGTQ,2021-12-28,NaN
...,...,...,...
4091,USDMXN,2023-10-20,18.2351
2045,USDGTQ,2023-10-20,7.8388
1022,USDBRL,2023-10-20,5.0345
3068,USDINR,2023-10-20,83.2396


In [19]:
# Sorting by date and country for proper forward filling
rates_cont = rates_cont.sort_values(by=['symbol', 'date'])

In [29]:
rates_cont.to_excel('checkNaN_rates.xlsx')

In [28]:
rates_cont[rates_cont['feed_price'].isnull()]

,symbol,date,feed_price
2,USDBRL,2021-01-03,NaN
7,USDBRL,2021-01-08,NaN
8,USDBRL,2021-01-09,NaN
11,USDBRL,2021-01-12,NaN
14,USDBRL,2021-01-15,NaN
...,...,...,...
5107,USDPHP,2023-10-13,NaN
5108,USDPHP,2023-10-14,NaN
5109,USDPHP,2023-10-15,NaN
5111,USDPHP,2023-10-17,NaN


In [ ]:
# Filling missing values downward within each country group
rates_cont['feed_price'] = rates_cont.groupby('symbol')['feed_price'].fillna(method='ffill')

In [41]:
rates_cont[rates_cont['date'] > '2021-12-26'].sort_values('date').head(20) # OK! 

,symbol,date,feed_price
360,USDBRL,2021-12-27,5.6254
3429,USDMXN,2021-12-27,20.7050
2406,USDINR,2021-12-27,74.9377
4452,USDPHP,2021-12-27,50.2775
1383,USDGTQ,2021-12-27,7.7211
2407,USDINR,2021-12-28,74.9377
3430,USDMXN,2021-12-28,20.7050
4453,USDPHP,2021-12-28,50.2775
361,USDBRL,2021-12-28,5.6254
1384,USDGTQ,2021-12-28,7.7211


In [42]:
rates_cont[rates_cont['date'] > '2023-01-22'].sort_values('date').head(20) # OK! 

,symbol,date,feed_price
752,USDBRL,2023-01-23,5.2082
2798,USDINR,2023-01-23,81.5729
3821,USDMXN,2023-01-23,18.8103
4844,USDPHP,2023-01-23,54.4597
1775,USDGTQ,2023-01-23,7.8526
753,USDBRL,2023-01-24,5.1398
2799,USDINR,2023-01-24,81.5724
3822,USDMXN,2023-01-24,18.8012
1776,USDGTQ,2023-01-24,7.8406
4845,USDPHP,2023-01-24,54.6760


In [35]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'last_daily_forex'

In [36]:
from pyathena import connect

#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df_forex = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_1356/3462653592.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_forex = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [37]:
df_forex

,symbol,max_feed_date,max_feed_price,day
0,USDMXN,2021-03-15 23:58:24,20.7702,2021-03-15
1,USDBRL,2021-03-15 23:58:24,5.6163,2021-03-15
2,USDPHP,2021-03-15 23:58:24,48.6090,2021-03-15
3,USDINR,2021-03-15 23:58:24,72.7402,2021-03-15
4,USDPHP,2021-12-06 23:58:27,50.4827,2021-12-06
...,...,...,...,...
5120,USDBRL,2023-08-25 23:58:27,4.8903,2023-08-25
5121,USDGTQ,2023-08-25 23:58:27,7.8744,2023-08-25
5122,USDMXN,2023-04-05 23:58:26,18.3364,2023-04-05
5123,USDGTQ,2023-05-22 23:58:29,7.8081,2023-05-22


,symbol,max_feed_date,max_feed_price,day
